# LangChain 学习手册 02：核心组件

本笔记本继续聚焦 LangChain v0.3.79，帮助你掌握 Prompt、LLM、输出解析器、Runnables 等核心概念。


## Prompt 模板与变量
LangChain 的 `ChatPromptTemplate` 支持多轮消息与变量注入。


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from datetime import datetime

prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '今天是 {date}。请以中文回答问题。'),
        ('human', '{question}')
    ]
)

formatted = prompt.invoke(
    {
        'date': datetime.utcnow().date().isoformat(),
        'question': '给我三条学习 LangChain 的建议'
    }
)

for message in formatted.messages:
    print(message.type, ':', message.content)


## Runnables：组合式流水线

`Runnable` 是 0.3 版本的核心抽象。你可以通过 `|` 运算符组合各个步骤，形成端到端的流程。


In [ ]:
            from langchain_community.chat_models.fake import FakeListChatModel
            from langchain_core.output_parsers import StrOutputParser
            from langchain_core.runnables import RunnableLambda

            model = FakeListChatModel(
                responses=['1. 阅读官方教程
2. 亲手实现示例
3. 构建自己的项目']
            )
            parser = StrOutputParser()

            def post_process(text: str) -> str:
                return text.upper()

            chain = prompt | model | parser | RunnableLambda(lambda text: post_process(text))

            print(chain.invoke({'date': '2024-01-01', 'question': '我要如何实践 LangChain？'}))


## 输出结构化数据
当你需要结构化输出时，可以利用 `PydanticOutputParser` 或者 `JsonOutputParser`。


In [ ]:
            from langchain_core.pydantic_v1 import BaseModel, Field
            from langchain.output_parsers.json import JsonOutputParser
            from langchain_core.prompts import PromptTemplate
            from langchain_community.llms.fake import FakeListLLM

            class StudyPlan(BaseModel):
                topic: str = Field(description='学习主题')
                steps: list[str] = Field(description='循序渐进的步骤')

            parser = JsonOutputParser(pydantic_object=StudyPlan)

            prompt = PromptTemplate(
                template='为主题 {topic} 设计一份学习计划，返回 JSON，字段为 topic 与 steps。
{format_instructions}',
                input_variables=['topic'],
                partial_variables={'format_instructions': parser.get_format_instructions()}
            )

            llm = FakeListLLM(
                responses=[
                    '{"topic": "LangChain", "steps": ["阅读文档", "动手写链", "集成工具"]}'
                ]
            )

            chain = prompt | llm | parser
            plan = chain.invoke({'topic': 'LangChain'})
            plan


## 小结
- Prompt 模板允许你组织上下文与变量。
- Runnables 让链路组合变得清晰、可测试。
- 输出解析器帮助你更可靠地获取结构化结果。
